In [3]:
import serial

port = '/dev/ttyUSB0'  # or COM3, adjust
baud = 115200

ser = serial.Serial(port, baud)
print("Connected to", ser.name)
print("Reading lines...")
while True:
    print(ser.readline().decode(errors='ignore').strip())


Connected to /dev/ttyUSB0
Reading lines...
-3676,76,15904,-279,135,-55,idle
-3648,40,15824,-289,149,-53,idle
-3696,48,15792,-327,147,-32,idle
-3680,76,15904,-355,158,-14,idle
-3668,156,15848,-383,122,-34,idle
-3688,80,15788,-359,140,-32,idle
-3776,80,15876,-343,164,-33,idle
-3648,160,15784,-309,196,-92,idle
-3716,40,15836,-284,169,-75,idle
-3628,16,15752,-285,132,-77,idle
-3660,100,15776,-297,144,-68,idle
-3728,48,15856,-295,143,-81,idle
-3628,84,15776,-262,152,-69,idle
-3668,20,15816,-366,176,-79,idle
-3592,104,15784,-297,107,-55,idle
-3648,28,15804,-295,148,-73,idle
-3632,68,15920,-293,165,-50,idle
-3640,76,15924,-326,163,-56,idle
-3696,84,15832,-337,144,-25,idle
-3660,112,15748,-306,134,-78,idle
-3664,80,15856,-310,153,-17,idle
-3676,156,15824,-289,164,-23,idle
-3648,128,15812,-340,137,-28,idle
-3656,52,15808,-308,135,-31,idle
-3664,76,15868,-281,182,-79,idle
-3676,88,15936,-285,148,-115,idle
-3696,-4,15776,-278,161,-45,idle
-3664,60,15856,-319,185,-13,idle
-3712,0,15820,-372,127,5,

KeyboardInterrupt: 

In [4]:
pip install pyserial

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.6/90.6 kB 1.5 MB/s eta 0:00:00? eta -:--:--
Note: you may need to restart the kernel to use updated packages.


In [5]:
import serial
import csv
from datetime import datetime

port = '/dev/ttyUSB0'  # change to your port
baud = 115200
label = 'idle' # change per session

with serial.Serial(port, baud, timeout=1) as ser, open(f'{label}.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(['timestamp', 'ax', 'ay', 'az', 'gx', 'gy', 'gz', 'label'])
    while True:
        line = ser.readline().decode().strip()
        if line:
            parts = line.split(',')
            if len(parts) == 6:
                ts = datetime.now().isoformat()
                writer.writerow([ts] + parts + [label])
                print(parts)


SerialException: device reports readiness to read but returned no data (device disconnected or multiple access on port?)

In [ ]:
import serial
import csv
import time
import sys
from datetime import datetime

sys.stdout.reconfigure(line_buffering=True)  # live printing

port = '/dev/ttyUSB0'
baud = 115200
label = 'idle'
num_samples = 1000

with serial.Serial(port, baud, timeout=1) as ser, open(f'{label}.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(['timestamp', 'ax', 'ay', 'az', 'gx', 'gy', 'gz', 'label'])
    
    print("Waiting for ESP32 to start...")
    time.sleep(3)
    print(f"Collecting {num_samples} samples for label '{label}'...")

    count = 0
    while count < num_samples:
        line = ser.readline().decode(errors='ignore').strip()
        if not line:
            continue
        parts = line.split(',')
        if len(parts) == 7:
            ts = datetime.now().isoformat()
            writer.writerow([ts] + parts)
            count += 1
            if count % 100 == 0:
                print(f"{count}/{num_samples} samples recorded", flush=True)
        time.sleep(0.01)

    print(f"✅ Done! Saved {num_samples} samples to {label}.csv", flush=True)
